In [405]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import warnings
import time

warnings.simplefilter('ignore')

In [409]:

def get_shareholding(x):
    url = "https://www.screener.in/company/" + x +"/"
    response = requests.get(url)

    soup = bs(response.content, "html.parser")

    quarterly_shp_section = soup.find('div', {'id': 'quarterly-shp'})

    shareholding_table = quarterly_shp_section.find('table', {'class': 'data-table'})
    shareholding_data = []
    for row in shareholding_table.find_all('tr'):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        shareholding_data.append(cols)

    df = pd.DataFrame(shareholding_data)
    time.sleep(4);
    return df

In [410]:

def HoldingScore(ticker):
    try:
        df = get_shareholding(ticker)
        if len(df[0].to_numpy()) == 6:
            df[0] = ["x","Promoter", "FII", "DII", "Retail","Share Holders"]
        if len(df[0].to_numpy()) == 7:
            df[0] = ["x","Promoter", "FII", "DII","Gov", "Retail","Share Holders"]
        if len(df[0].to_numpy()) == 8:
            df[0] = ["x","Promoter", "FII", "DII","Gov", "Retail","other","Share Holders"]
        
        df = df.transpose()

        df = df.drop(columns=[0])
        df.columns = df.iloc[0]
        df = df.iloc[1:]

        df.drop([1, 2, 3, 4, 5, 6, 7, 8], inplace=True)
        # Convert percentage strings to float
        df['Promoter'] = df['Promoter'].str.rstrip('%').astype(float)
        df['FII'] = df['FII'].str.rstrip('%').astype(float)
        df['DII'] = df['DII'].str.rstrip('%').astype(float)
        df['Retail'] = df['Retail'].str.rstrip('%').astype(float)
        try:
            df['Gov'] = df['Gov'].str.rstrip('%').astype(float)
        except:
            pass
        # Remove commas and convert 'share holders' to int
        df['Share Holders'] = df['Share Holders'].str.replace(',', '').astype(int)
        score = round((np.diff(df['FII'].to_numpy()) / df['FII'].to_numpy()[:-1]).sum() + (np.diff(df['DII'].to_numpy()) / df['DII'].to_numpy()[:-1]).sum() + (np.diff(df['Promoter'].to_numpy()) / df['Promoter'].to_numpy()[:-1]).sum() - (np.diff(df['Retail'].to_numpy()) / df['Retail'].to_numpy()[:-1]).sum(),5)
        holding = round(-(np.diff(df['Share Holders'].to_numpy()) / df['Share Holders'].to_numpy()[:-1]).sum(),3)
        return score, holding
    except :
        print('error '+ticker)
        return 0, 0

In [411]:
def csv_caller(x):
    of = pd.read_csv(x)
    of = of.drop(columns=['Unnamed: 0'])
    of["Holding score"] = 0
    of['Holding no'] = 0
    k = 0
    for i in of['nsecode']:
        Score, holding = HoldingScore(i)
        of['Holding score'][k] = Score
        of['Holding no'][k] = holding
        k+=1
    return of

In [412]:

df = csv_caller("long_term.csv")
df.to_csv("long.csv")
df = csv_caller("short_term.csv")
df.to_csv("short.csv")
df = csv_caller("mid_term.csv")
df.to_csv("mid.csv")


error CONSOFINVT
error CONSOFINVT
error SARLAPOLY
error DBOL


In [413]:
df

,nsecode,Holding score,Holding no
0,GARFIBRES,0.41239,0.043
1,BAJAJFINSV,0.43342,-0.226
2,NOVARTIND,0.32802,-0.002
3,BALRAMCHIN,0.15363,0.146
4,GRANULES,0.49594,0.002
5,KNRCON,-0.06711,-0.027
6,DBOL,0.00000,0.000


In [407]:
df = get_shareholding("M&M")

In [408]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Promoters +,19.58%,19.44%,19.45%,19.46%,19.46%,19.47%,19.47%,19.45%,19.38%,19.39%,19.37%,19.37%
2,FIIs +,34.72%,37.90%,38.93%,40.05%,38.92%,38.66%,37.30%,37.94%,38.27%,39.16%,39.24%,40.14%
3,DIIs +,30.68%,28.24%,27.35%,26.12%,27.08%,27.81%,28.78%,28.93%,28.50%,27.72%,27.66%,26.89%
4,Government +,0.24%,0.24%,0.24%,0.07%,0.07%,0.07%,0.07%,0.07%,0.07%,0.07%,0.07%,0.07%
5,Public +,10.56%,10.02%,9.91%,10.20%,10.40%,9.95%,10.40%,9.66%,9.83%,9.77%,9.81%,9.71%
6,Others +,4.22%,4.15%,4.12%,4.10%,4.07%,4.03%,3.97%,3.95%,3.92%,3.88%,3.85%,3.83%
7,No. of Shareholders,"4,27,354","4,10,346","4,44,750","5,31,369","5,81,779","5,68,603","6,32,816","5,81,288","6,49,341","6,36,956","6,47,141","6,36,984"


In [406]:
time.sleep(4)

In [414]:
long_ga = pd.read_csv("long.csv") 
mid_ga = pd.read_csv("mid.csv") 
short_ga = pd.read_csv("short.csv") 

In [415]:
long_ga

,Unnamed: 0,nsecode,Holding score,Holding no
0,0,BAJFINANCE,0.06494,-0.036
1,1,BAJAJ-AUTO,0.01031,0.102
2,2,ROLEXRINGS,0.00000,-0.000
3,3,ECLERX,0.16263,-0.398
4,4,M&M,0.00255,0.019
5,5,INDUSINDBK,0.02296,-0.022
6,6,BHARTIARTL,0.08108,0.091
7,7,GESHIP,-0.00193,-0.158
8,8,ICICIPRULI,0.18346,0.007
9,9,360ONE,3.04336,-0.283
